# Supervisor (Polyglot)

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.2.2/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common
open SpiralFileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = SpiralNetworking.test_port_open port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.Length: {result |> Option.ofObj |> Option.map String.length}") getLocals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> SpiralSm.format_exception}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let repositoryRoot = SpiralFileSystem.get_repository_root ()

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let struct (ct, disposable) = cancellationToken |> SpiralThreading.new_disposable_token
    let! ct = ct |> SpiralAsync.merge_cancellation_token_with_default_async

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = SpiralNetworking.get_available_port (Some 60) port
        if availablePort <> port then
            inbox.Post (port, false)
        else
            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let command = $@"dotnet ""{dllPath}"" --port {availablePort} --default-int i32 --default-float f64"
            let onLine = fun struct (_, line, _) -> async {
                if line |> SpiralSm.contains $"Server bound to: http://localhost:{availablePort}" then
                    do! SpiralNetworking.wait_for_port_access (Some 500) true availablePort |> Async.Ignore

                    let rec loop retry = async {
                        let getLocals () = $"port: {availablePort} / retry: {retry} / {getLocals ()}"
                        try
                            let pingObj = {| Ping = true |}
                            let! pingResult = pingObj |> sendObj availablePort
                            trace Verbose (fun () -> $"awaitCompiler / Ping / result: {pingResult}") getLocals
                        with ex ->
                            trace Verbose (fun () -> $"awaitCompiler / Ping / ex: {ex |> SpiralSm.format_exception}") getLocals
                            do! Async.Sleep 10
                            do! loop (retry + 1)
                    }
                    do! loop 0
                    inbox.Post (availablePort, true)
            }

            let! exitCode, result =
                SpiralRuntime.execute_with_options_async struct (Some ct, command, Some onLine, Some repositoryRoot)

            trace Debug (fun () -> $"awaitCompiler / exitCode: {exitCode} / result: {result}") getLocals
            disposable.Dispose ()
    }, ct)

    let! serverPort, managed = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable' = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable' =
        new_disposable (fun () ->
            async {
                disposable'.Dispose ()
                do! connection.StopAsync () |> Async.AwaitTask
                disposable.Dispose ()
                if managed
                then do! SpiralNetworking.wait_for_port_access (Some 2000) false serverPort |> Async.Ignore
            }
            |> Async.RunSynchronously
        )

    return
        serverPort,
        stream,
        ct,
        disposable'
}

## getFileUri

In [ ]:
let inline getFileUri (path : string) =
    let path =
        if SpiralRuntime.is_windows () |> not
        then path
        else $"{path.[0] |> System.Char.ToLower}{path.[1..]}" |> SpiralSm.replace "\\" "/"
    $"file:///{path |> SpiralSm.trim_start [| '/' |]}"

## getFilePathFromUri

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

## getCompilerPort

In [ ]:
let inline getCompilerPort () =
    13805

## serialize_obj

In [ ]:
    let serializeObj obj =
        obj
        |> FSharp.Json.Json.serialize
        |> SpiralSm.replace "\\\\" "\\"
        |> SpiralSm.replace "\\r\\n" "\n"
        |> SpiralSm.replace "\\n" "\n"

## buildFile

In [ ]:
let inline buildFile timeout port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> SpiralFileSystem.read_all_text_async

    let stream, disposable = fileDir |> FileSystem.watchDirectory (fun _ -> false)
    use _ = disposable

    let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
    use _ = disposable

    let! serverPort, errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.chooseAsync (function
            | _, (FileSystem.FileSystemChange.Changed (path, _))
                when (path |> System.IO.Path.GetFileName) = $"{fileName}.fsx"
                ->
                    fileDir </> path |> SpiralFileSystem.read_all_text_retry_async
            | _ -> None |> Async.init
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> SpiralSm.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> SpiralSm.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, error)
            | TracedError data ->
                Some (data.message, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, error) ->
            None, Some (message, error)
        )

    let timerSeq =
        1000
        |> FSharp.Control.AsyncSeq.intervalMs
        |> FSharp.Control.AsyncSeq.map (fun _ -> None, None)

    let outputSeq =
        [ fsxContentSeq; errorsSeq; timerSeq ]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], 0), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, typeErrorCount) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, typeErrorCount
                | None, Some (_, FatalError "File main has a type error somewhere in its path.") ->
                    fsxContentResult, errors, typeErrorCount + 1
                | None, Some error -> fsxContentResult, error :: errors, typeErrorCount
                | None, None when typeErrorCount >= 1 ->
                    fsxContentResult, errors, typeErrorCount + 1
                | _ -> fsxContentResult, errors, typeErrorCount
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, typeErrorCount) ->
            trace Debug (fun () -> $"buildFile / takeWhileInclusive / fsxContent: {fsxContent |> Option.defaultValue System.String.Empty |> SpiralSm.ellipsis_end 400} / errors: {errors |> serializeObj} / typeErrorCount: {typeErrorCount}") getLocals
#if INTERACTIVE
            let errorWait = 2
#else
            let errorWait = 5
#endif
            match fsxContent, errors with
            | None, [] when typeErrorCount > errorWait -> false
            | None, [] -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.withCancellationToken ct
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    do! Async.Sleep 60

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some (message, errors, _))) ->
                message, errors |> List.distinct |> List.rev
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> serializeObj}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode code = async {
    let targetDir = repositoryRoot </> "target/polyglot/spiral_eval"

    let packagesDir = targetDir </> "packages"

    let hashHex = code |> SpiralCrypto.hash_text

    let codeDir = packagesDir </> hashHex

    codeDir |> System.IO.Directory.CreateDirectory |> ignore

    let mainPath = codeDir </> "main.spi"
    do! code |> SpiralFileSystem.write_all_text_exists mainPath

    let spiprojPath = codeDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "lib"}
packages:
    |core-
    spiral-
modules:
    main
"""
    do! spiprojCode |> SpiralFileSystem.write_all_text_exists spiprojPath

    return mainPath
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath = persistCode code
    let port = getCompilerPort ()
    return! mainPath |> buildFile timeout port cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"""inl app () =
    console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 15000 None
|> Async.runWithTimeout 15000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:06 verbose #1 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:06 debug #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:07 verbose #2 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:06 verbose #2 > pwd: /home/runner/work/polyglot/polyglot
00:00:06 verbose #3 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:06 verbose #4 > targetDir: /home/runner/work/polyglot/polyglot/target/polyglot/spiral_eval
00:00:07 verbose #3 wait_for_port_access / port: 138

In [ ]:
//// test

""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> _assertEqual None

00:00:09 verbose #32 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 debug #8 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:09 verbose #9 > pwd: /home/runner/work/polyglot/polyglot
00:00:10 verbose #33 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:10 verbose #34 wait_for_port_access / port: 13805 / retry: 0
00:00:09 verbose #10 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:09 verbose #11 > targetDir: /home/runner/work/polyglot/polyglot/target/po

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "main.spi:
Global inl/let statements should all return functions known at parse time." ]
    )
)

00:00:19 verbose #61 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:19 debug #18 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:19 verbose #19 > pwd: /home/runner/work/polyglot/polyglot
00:00:20 verbose #62 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:20 verbose #63 wait_for_port_access / port: 13805 / retry: 0
00:00:19 verbose #20 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:19 verbose #21 > targetDir: /home/runner/work/polyglot/polyglot/target/

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "An attempt to divide by zero has been detected at compile time." ]
    )
)

00:00:21 verbose #90 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:20 debug #25 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:20 verbose #26 > pwd: /home/runner/work/polyglot/polyglot
00:00:21 verbose #91 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:21 verbose #92 wait_for_port_access / port: 13805 / retry: 0
00:00:20 verbose #27 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:20 verbose #28 > targetDir: /home/runner/work/polyglot/polyglot/target/

In [ ]:
//// test

"""inl main () =
    1 + ""
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Constraint satisfaction error.
Got: string
Fails to satisfy: number"
        ]
    )
)

00:00:23 verbose #119 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:22 debug #32 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:22 verbose #33 > pwd: /home/runner/work/polyglot/polyglot
00:00:23 verbose #120 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:23 verbose #121 wait_for_port_access / port: 13805 / retry: 0
00:00:22 verbose #34 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:22 verbose #35 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""inl main () =
    x + y
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Unbound variable: x.
Unbound variable: y."
        ]
    )
)

00:00:25 verbose #149 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:24 debug #39 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:25 verbose #40 > pwd: /home/runner/work/polyglot/polyglot
00:00:25 verbose #150 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:25 verbose #151 wait_for_port_access / port: 13805 / retry: 0
00:00:25 verbose #41 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:25 verbose #42 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "main.spi:
Expected: uppercase variable" ]
    )
)

00:00:27 verbose #177 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:26 debug #46 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:27 verbose #47 > pwd: /home/runner/work/polyglot/polyglot
00:00:28 verbose #178 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:28 verbose #179 wait_for_port_access / port: 13805 / retry: 0
00:00:27 verbose #48 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:27 verbose #49 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "main.spi:
Expected: whitespace" ]
    )
)

00:00:29 verbose #206 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:28 debug #53 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:28 verbose #54 > pwd: /home/runner/work/polyglot/polyglot
00:00:29 verbose #207 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:29 verbose #208 wait_for_port_access / port: 13805 / retry: 0
00:00:28 verbose #55 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:28 verbose #56 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "Cannot apply a forall with a term." ]
    )
)

00:00:30 verbose #234 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:29 debug #59 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:29 verbose #60 > pwd: /home/runner/work/polyglot/polyglot
00:00:30 verbose #235 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:30 verbose #236 wait_for_port_access / port: 13805 / retry: 0
00:00:29 verbose #61 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:29 verbose #62 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "The main function should not have a forall." ]
    )
)

00:00:32 verbose #262 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:31 debug #66 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:31 verbose #67 > pwd: /home/runner/work/polyglot/polyglot
00:00:32 verbose #263 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:32 verbose #264 wait_for_port_access / port: 13805 / retry: 0
00:00:31 verbose #68 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:31 verbose #69 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:34 verbose #289 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:33 debug #73 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:34 verbose #74 > pwd: /home/runner/work/polyglot/polyglot
00:00:35 verbose #290 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:35 verbose #291 wait_for_port_access / port: 13805 / retry: 0
00:00:34 verbose #75 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:34 verbose #76 > targetDir: /home/runner/work/polyglot/polyglot/targ

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some "0.33332500000000004\n",
        []
    )
)
// |> _assertEqual None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

00:00:37 verbose #317 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:36 debug #80 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:buildCode@3-314>, Some "/home/runner/work/polyglot/polyglot")
00:00:36 verbose #81 > pwd: /home/runner/work/polyglot/polyglot
00:00:37 verbose #318 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:37 verbose #319 wait_for_port_access / port: 13805 / retry: 0
00:00:36 verbose #82 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:36 verbose #83 > targetDir: /home/runner/work/polyglot/polyglot/targ

## getFileTokenRange

In [ ]:
let inline getFileTokenRange port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> SpiralFileSystem.read_all_text_async
    let lines = code |> SpiralSm.split "\n"

    let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
    use _ = disposable

    let! serverPort, _errors, ct, disposable =
        match port with
        | Some port -> awaitCompiler port (Some token)
        | None -> (getCompilerPort (), FSharp.Control.AsyncSeq.empty, token, new_disposable id) |> Async.init
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult =
        fileTokenRangeObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange cancellationToken code = async {
    let! mainPath = persistCode code
    let port = getCompilerPort ()
    return! mainPath |> getFileTokenRange (Some port) cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0 |])

00:00:40 verbose #345 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:39 debug #87 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:it@4-196>, Some "/home/runner/work/polyglot/polyglot")
00:00:40 verbose #88 > pwd: /home/runner/work/polyglot/polyglot
00:00:40 verbose #89 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:40 verbose #90 > targetDir: /home/runner/work/polyglot/polyglot/target/polyglot/spiral_eval
00:00:41 verbose #346 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:41 verbose #347 wait_for_port_access / port: 

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0 |])

00:00:42 verbose #374 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:41 debug #93 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:it@4-343>, Some "/home/runner/work/polyglot/polyglot")
00:00:41 verbose #94 > pwd: /home/runner/work/polyglot/polyglot
00:00:41 verbose #95 > dllPath: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:41 verbose #96 > targetDir: /home/runner/work/polyglot/polyglot/target/polyglot/spiral_eval
00:00:42 verbose #375 test_port_open / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:42 verbose #376 wait_for_port_access / port: 

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | Build_File of string * string
    | File_Token_Range of string * string
    | Execute_Command of string
    | [<Argu.ArguAttributes.Unique>] Timeout of int
    | [<Argu.ArguAttributes.Unique>] Port of int
    | [<Argu.ArguAttributes.Unique>] Parallel

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Build_File _ -> nameof Build_File
            | File_Token_Range _ -> nameof File_Token_Range
            | Execute_Command _ -> nameof Execute_Command
            | Timeout _ -> nameof Timeout
            | Port _ -> nameof Port
            | Parallel -> nameof Parallel

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

USAGE: dotnet-repl [--help] [--build-file <string> <string>]
                   [--file-token-range <string> <string>]
                   [--execute-command <string>] [--timeout <int>] [--port <int>]
                   [--parallel]

OPTIONS:

    --build-file <string> <string>
                          Build_File
    --file-token-range <string> <string>
                          File_Token_Range
    --execute-command <string>
                          Execute_Command
    --timeout <int>       Timeout
    --port <int>          Port
    --parallel            Parallel
    --help                display this list of options.


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let buildFileActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Build_File)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Build_File (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileTokenRangeActions =
        argsMap
        |> Map.tryFind (nameof Arguments.File_Token_Range)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.File_Token_Range (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let executeCommandActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Execute_Command)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Execute_Command command -> Some command
            | _ -> None
        )

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 60000 * 60

    let port =
        match argsMap |> Map.tryFind (nameof Arguments.Port) with
        | Some [ Arguments.Port port ] -> Some port
        | _ -> None

    let isParallel = argsMap |> Map.containsKey (nameof Arguments.Parallel)

    async {
        let port = port |> Option.defaultWith getCompilerPort
        let struct (localToken, disposable) = SpiralThreading.new_disposable_token None
        let! serverPort, _errors, compilerToken, disposable = awaitCompiler port (Some localToken)
        use _ = disposable

        let buildFileAsync =
            buildFileActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let! outputCode, errors = inputPath |> buildFile timeout serverPort None

                errors
                |> List.map snd
                |> List.iter (fun error ->
                    trace Critical (fun () -> $"main / error: {error |> serializeObj}") getLocals
                )

                match outputCode with
                | Some outputCode ->
                    do! outputCode |> SpiralFileSystem.write_all_text_async outputPath
                    return 0
                | None ->
                    return 1
            })

        let fileTokenRangeAsync =
            fileTokenRangeActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let! tokenRange = inputPath |> getFileTokenRange (Some serverPort) None
                match tokenRange with
                | Some tokenRange ->
                    do! tokenRange |> FSharp.Json.Json.serialize |> SpiralFileSystem.write_all_text_async outputPath
                    return 0
                | None ->
                    return 1
            })

        let executeCommandAsync =
            executeCommandActions
            |> List.map (fun command -> async {
                let! exitCode, result =
                    SpiralRuntime.execute_with_options_async struct (Some compilerToken, command, None, None)

                trace Debug (fun () -> $"main / executeCommand / exitCode: {exitCode}") getLocals

                return exitCode
            })

        return!
            [| buildFileAsync; fileTokenRangeAsync; executeCommandAsync |]
            |> Seq.collect id
            |> fun x ->
                if isParallel
                then Async.Parallel (x, float System.Environment.ProcessorCount * 0.51 |> ceil |> int)
                else Async.Sequential x
            |> Async.map Array.sum
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0